In [1]:
import fcalc
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
import time
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('data_sets/iris.data', names=['sepal_length',	'sepal_width',	'petal_length',	'petal_width','species'])
le = LabelEncoder(); le.fit(df["species"].values)
df["species"] = le.transform(df["species"].values)

In [3]:
def model_test_CV(X, y, cat_cols=None, method="standard", alpha=0.,
                  randomize=False, num_iters=10, subsample_size=1e-2,
                  kde_bandwidth=1.0, kde_kernel='gaussian', kde_leaf_size=40,
                  kde_classwise=False, scale_density=True, n_splits=5, seed=42):
    kf = KFold(n_splits=n_splits, random_state=seed, shuffle=True)
    Accuracy = []
    F1_score = []
    exec_time = []
    for train_index, test_index in tqdm(kf.split(X), total=kf.get_n_splits(), desc="k-fold"):
        pat_cls = fcalc.classifier.PatternClassifier(X[train_index], y[train_index], 
                                                     categorical=cat_cols, method=method,
                                                     alpha=alpha, randomize=randomize, num_iters=num_iters,
                                                     subsample_size=subsample_size, kde_bandwidth=kde_bandwidth,
                                                     kde_kernel=kde_kernel, kde_leaf_size=kde_leaf_size,
                                                     kde_classwise=kde_classwise, scale_density=scale_density)
        start = time.time()
        pat_cls.predict(X[test_index])
        end = time.time()
        Accuracy.append(round(accuracy_score(y[test_index], pat_cls.predictions),4))
        F1_score.append(round(f1_score(y[test_index], pat_cls.predictions, average='macro'),4))
        exec_time.append(round(end-start, 4))
    
    Accuracy.append(np.mean(Accuracy)); F1_score.append(np.mean(F1_score)); exec_time.append(np.mean(exec_time))
    return pd.DataFrame(zip(Accuracy,F1_score,exec_time),
                        columns=["Accuracy","F1 score", "time (s)"],
                        index=[x+1 for x in range(kf.get_n_splits())]+["mean"])

In [5]:
X_train = df.drop('species', axis=1).values
y_train = df['species'].values
n_iters = np.arange(10,51,10)
s_size = np.arange(1,11)#np.linspace(1e-2,5e-2,num=5,endpoint=True)
result = []
for n in n_iters:
    for s in s_size:
        res = model_test_CV(X_train,y_train,randomize=True,num_iters=n,subsample_size=s)
        result.append(res.loc["mean"].values)
result=pd.DataFrame(result,columns=["Accuracy","F1 score", "time (sec.)"], 
                    index=pd.MultiIndex.from_product([n_iters, s_size], names=["Number of iterations","Subsample size"]))
display(result)
result.to_csv("D:/University/masters thesis/csv results/iris_random_res_multiindex.csv", index=True)

k-fold: 100%|██████████| 5/5 [00:00<00:00,  6.33it/s]


Accuracy  F1 score  time (sec.)
Number of iterations Subsample size                                 
10                   1                0.82668   0.66740      0.03268
                     2                0.90666   0.73390      0.03122
                     3                0.92000   0.84396      0.02970
                     4                0.90000   0.73046      0.02978
                     5                0.88000   0.79842      0.03036
                     6                0.91332   0.82794      0.02926
                     7                0.83336   0.72634      0.03380
                     8                0.91332   0.90798      0.03580
                     9                0.81334   0.69874      0.03874
                     10               0.89998   0.86228      0.03380
20                   1                0.86000   0.67932      0.06458
                     2                0.91334   0.78500      0.06200
                     3                0.91334   0.82500      0.06774
                     4                0.90000   0.80966      0.06164
                     5                0.90000   0.89372      0.06168
                     6                0.90668   0.81744      0.06128
                     7                0.90666   0.77204      0.05960
                     8                0.90666   0.86032      0.05778
                     9                0.82668   0.75478      0.06036
                     10               0.90666   0.90096      0.05826
30                   1                0.90666   0.75190      0.08736
                     2                0.91334   0.82542      0.08562
                     3                0.92668   0.83984      0.09040
                     4                0.90000   0.76678      0.08630
                     5                0.90000   0.89372      0.08820
                     6                0.90668   0.86010      0.08884
                     7                0.90666   0.90074      0.08656
                     8                0.90666   0.81798      0.09288
                     9                0.90000   0.89372      0.09106
                     10               0.90666   0.85842      0.08722
40                   1                0.91332   0.75214      0.11796
                     2                0.92000   0.82988      0.12774
                     3                0.93332   0.88592      0.12460
                     4                0.89332   0.84832      0.12262
                     5                0.90000   0.89372      0.11722
                     6                0.90000   0.76616      0.11994
                     7                0.92000   0.91448      0.11882
                     8                0.91332   0.86754      0.11548
                     9                0.90000   0.89372      0.11620
                     10               0.90666   0.90074      0.12054
50                   1                0.91332   0.75214      0.14798
                     2                0.92000   0.83454      0.14940
                     3                0.92668   0.92314      0.14420
                     4                0.89332   0.80634      0.14400
                     5                0.90000   0.89372      0.14930
                     6                0.90666   0.86030      0.14626
                     7                0.91334   0.86626      0.14752
                     8                0.90666   0.86032      0.15070
                     9                0.90666   0.90074      0.15192
                     10               0.90000   0.85118      0.15542

OSError: Cannot save file into a non-existent directory: 'D:\University\masters thesis\csv results'